In [1]:
import warnings
import numpy as np
import scipy.interpolate
import astropy.units as u
import matplotlib.pyplot as plt
import fiasco

%matplotlib inline

# Testing an Ion Collection Object
Need to have an object that combines several ions into a collection. This is most useful for calculating multi-ion quantities like spectra and radiative losses.

What are the ways we could create an `IonCollection`?

1. Instantiate an `IonCollection` object with
  - a list of ion strings, e.g. `['fe_1','ca_7','li_2']`
  - a list of `Ion` objects, e.g. `[Ion('fe_1'),Ion('ca_7'),Ion('li_2')]`
  - a list of element strings, e.g. `['Fe','Ca']`
  - a list of `Element` objects, e.g. `[Element('Fe'),Element('Ca')]`
  - any combination of the above
2. Add two or more `Element` or `Ion` objects together, e.g.
  - `Element('Fe') + Element('Ca')`
  - `Ion('fe_1') + Ion('ca_7')`
3. Add two or more `IonCollection` objects together
  
This make things like spectra and radiative losses easily "composable" and is very intuitive for the user, particularly the latter approach. 

A few notes:
- To have two or more ions in an `IonCollection`, the temperature and densities **must be the same**.
- If only a string is included anywhere in the list, a temperature must also be specified
- When an element is added to a collection, it is expanded into its component ions, i.e. there is no need to have a separate collection for elements
- We need to filter out duplicates such that if I include both `Element('Fe')` and `Ion('fe_2')`, I don't have Fe II in the collection twice


In [2]:
temperature = np.logspace(4,9,100)*u.K

In [ ]:
class IonCollection(object):
    
    def __init__(self,*args):
        self._ion_list = []
        for item in args:
            if isinstance(item, fiasco.Ion):
                self._ion_list.append(item)
            elif isinstance(item, fiasco.Element):
                self._ion_list += [ion for ion in item]
            elif isinstance(item, type(self)):
                self._ion_list += item._ion_list
            else:
                raise TypeError('{} has an unrecognized type and cannot be added to collection.'.format(item))
        # TODO: check for duplicates
        # TODO: check all temperatures are the same
        
    def __getitem__(self,x):
        return self._ion_list[x]
    
    def __contains__(self,x):
        return x in [i.ion_name for i in self._ion_list]
    
    def __add__(self,x):
        return IonCollection(self._ion_list+[x])
    
    def __radd__(self,x):
        return IonCollection([x]+self._ion_list)

Test direct instantiation

In [3]:
el1 = fiasco.Element('Fe',temperature)
el2 = fiasco.Element('ca',temperature)
ion1 = fiasco.Ion('li_2',temperature)
ion2 = fiasco.Ion('li_1',temperature)
ionb = fiasco.IonBase('li_2')

In [8]:
c12 = el1+el2+ion1+ion2

In [9]:
'Fe 26' in c12

True

In [13]:
ion1.ip.decompose().cgs

<Quantity 1.211888881964882e-10 erg>

In [ ]:
plt.plot(ion1.temperature,ion1.ioneq)
plt.plot(ionb.ioneq['chianti']['temperature'],ionb.ioneq['chianti']['ionization_fraction'],'.')
plt.plot(ion2.temperature,ion2.ioneq)
plt.xscale('log')

In [ ]:
ionb.ioneq

In [ ]:
f = scipy.interpolate.interp1d(ionb.ioneq['shull_steenberg']['temperature'],
                               ionb.ioneq['shull_steenberg']['ionization_fraction'],
                               kind='cubic')
plt.plot(ionb.ioneq['shull_steenberg']['temperature'],ionb.ioneq['shull_steenberg']['ionization_fraction'],'.')
plt.plot(temperature,f(temperature.value))
plt.xscale('log')

In [ ]:
ic = IonCollection(el1,el2,ion1,ion2)

In [ ]:
for ion in ic:
    print(ion.ion_name)

In [ ]:
'li_3' in ic

In [ ]:
ic2 = IonCollection('fe_2',temperature=temperature)

In [ ]:
ic2[0].ion_name

Testing composable approach

In [ ]:
class TestIon(fiasco.Ion):
    def __add__(self,x):
        return IonCollection([self.ion_name,x],temperature=self.temperature)
    
    def __radd__(self,x):
        return IonCollection([x,self.ion_name],temperature=self.temperature)

class TestElement(fiasco.Element):
    def __add__(self,x):
        return IonCollection([self.element_name,x],temperature=self.temperature)
    def __radd__(self,x):
        return IonCollection([x,self.element_name],temperature=self.temperature)

In [ ]:
tion1 = TestIon('li_2',temperature)
tion2 = TestIon('he_2',temperature)
tion3 = TestIon('fe_18',temperature)

In [ ]:
ic3 = tion1 + tion2 + tion3 + ic

In [ ]:
for ion in ic3:
    print(ion.ion_name)

In [ ]:
foo1 = [1,2,3]
foo2 = 4
foo(*([foo2] + foo1))